In [340]:
import requests
import xml.etree.ElementTree as ET
import time
import json
import difflib
from collections import namedtuple
from urllib.parse import quote, unquote
from openpyxl import Workbook, load_workbook

In [257]:
# read the request "getItem" info
rq_info = str() # request info
rq_info_path = "getItem.json"
with open(rq_info_path, "r", encoding = "utf-8") as file:
    rq_info = file.read()
    rq_info = json.loads(rq_info)

# extract data from the rq_info
URL = rq_info["url"]
headers = dict()
for data in rq_info["headers"]:
    headers[data["name"]] = data["value"]
    
postData = rq_info["postData"]["params"]
postDataStr = ""
if(isinstance(postData, list)):
    for i, post in enumerate(postData):
        if(i != 0):
            postDataStr = postDataStr + '&'
        postDataStr = postDataStr + post["name"] + '=' + post["value"]
    postData = postDataStr

In [314]:
from random import random
random()


0.9438946936782628

In [107]:
itemGetter = requests.post(url = URL, headers = headers, data = postData)

In [238]:
root = ET.fromstring(itemGetter.text)

In [251]:
itemCode_dict = dict()
for child in root:
    data = child.attrib
    itemCode_dict[data["Label"].strip()] = data["Handle"]
itemCode_dict
print(json.dumps(itemCode_dict))

<class 'str'>


In [115]:
with open("./items.xml", 'w', encoding="utf-8") as file:
    file.write(itemGetter.text)

In [113]:
root = ET.fromstring(itemGetter.text)

In [131]:
Item = namedtuple("Items", ["ItemCode"])
itemDict = dict()
for child in root:
    itemDict[child.attrib["Label"].lower().strip()] = Item(ItemCode = child.attrib["_ItemCode"])

In [140]:
itemDict["japan"]

Items(ItemCode='392')

In [333]:
# read the request "getData" info
rq_info = str() # request info
rq_info_path = "./config/getData.json"
with open(rq_info_path, "r", encoding = "utf-8") as file:
    rq_info = file.read()
    rq_info = json.loads(rq_info)

# extract data from the rq_info
URL = rq_info["url"]
headers = dict()
for data in rq_info["headers"]:
    headers[data["name"]] = data["value"]
    
postData = dict()
for i, post in enumerate(rq_info["postData"]["params"]):
    postData[post["name"]] = post["value"]

postData["sWD_ReportId"] = "180622"
postData["sWD_TableId"] = "1494"

In [334]:
print(postData)

{'sWD_ReportId': '180622', 'sWD_TableId': '1494', 'sWD_ReportView': '<ReportView><RowDims><Dim+name=%22ECONOMY%22><Groups+activeGroup=%22e612da50-1b0a-4a2e-ab06-7431c214e37a%22><Group+id=%22e612da50-1b0a-4a2e-ab06-7431c214e37a%22+type=%22Selection%22><Definition><Items+type=%22handles%22><String+value=%220,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,219%22%2F><%2FItems><%2FDefinition><ActiveItem+pos=%221%22+%2F><%2FGroup><%2

In [335]:
reportView = unquote(postData["sWD_ReportView"])
reportView = '<?xml version="1.0" encoding="UTF-8"?>' + reportView
reportView = reportView.replace('+', ' ')
reportView = ET.fromstring(reportView)
for target in reportView.findall(r"./OtherDims/Dim[@name='PARTNER']"):
    target.find(".//String").attrib["value"] = "48" # 一次只能一個喔
    
for target in reportView.findall(r"./RowDims/Dim[@name='ECONOMY']"):
    target.find(".//String").attrib["value"] = "219" # 一次只能一個喔
    
for target in reportView.findall(r"./OtherDims/Dim[@name='PRODUCT']"):
    target.find(".//ActiveItem").attrib["pos"] = "0" # 一次只能一個喔
    
reportView = ET.tostring(reportView).decode("utf-8")
print(reportView)

<ReportView><RowDims><Dim name="ECONOMY"><Groups activeGroup="e612da50-1b0a-4a2e-ab06-7431c214e37a"><Group id="e612da50-1b0a-4a2e-ab06-7431c214e37a" type="Selection"><Definition><Items type="handles"><String value="219" /></Items></Definition><ActiveItem pos="1" /></Group></Groups></Dim></RowDims><ColDims><Dim name="YEAR"><Groups activeGroup="0e0da7b2-0d35-400d-ab5a-b4f842cf34c2"><Group id="0e0da7b2-0d35-400d-ab5a-b4f842cf34c2" type="Selection"><Definition><All /></Definition><ActiveItem pos="0" /></Group></Groups></Dim></ColDims><OtherDims><Dim name="PRODUCT"><Groups activeGroup="8b289558-d1fc-4c95-9280-5743a29e49f5"><Group id="8b289558-d1fc-4c95-9280-5743a29e49f5" type="Selection"><Definition><All /></Definition><ActiveItem pos="0" /></Group></Groups></Dim><Dim name="PARTNER"><Groups activeGroup="1509a964-c115-4299-868d-eee701fabaca"><Group id="1509a964-c115-4299-868d-eee701fabaca" type="Selection"><Definition><Items type="handles"><String value="48" /></Items></Definition><ActiveIte

In [336]:
postData["sWD_ReportView"] = reportView
postDataStr = ""
count = 0
for key, value in postData.items():
    if(count != 0):
        postDataStr = postDataStr + '&'
    postDataStr = postDataStr + key + '=' + value
    count += 1
print(postDataStr)

sWD_ReportId=180622&sWD_TableId=1494&sWD_ReportView=<ReportView><RowDims><Dim name="ECONOMY"><Groups activeGroup="e612da50-1b0a-4a2e-ab06-7431c214e37a"><Group id="e612da50-1b0a-4a2e-ab06-7431c214e37a" type="Selection"><Definition><Items type="handles"><String value="219" /></Items></Definition><ActiveItem pos="1" /></Group></Groups></Dim></RowDims><ColDims><Dim name="YEAR"><Groups activeGroup="0e0da7b2-0d35-400d-ab5a-b4f842cf34c2"><Group id="0e0da7b2-0d35-400d-ab5a-b4f842cf34c2" type="Selection"><Definition><All /></Definition><ActiveItem pos="0" /></Group></Groups></Dim></ColDims><OtherDims><Dim name="PRODUCT"><Groups activeGroup="8b289558-d1fc-4c95-9280-5743a29e49f5"><Group id="8b289558-d1fc-4c95-9280-5743a29e49f5" type="Selection"><Definition><All /></Definition><ActiveItem pos="0" /></Group></Groups></Dim><Dim name="PARTNER"><Groups activeGroup="1509a964-c115-4299-868d-eee701fabaca"><Group id="1509a964-c115-4299-868d-eee701fabaca" type="Selection"><Definition><Items type="handles">

In [337]:
dataGetter = requests.post(url = URL, headers = headers, data = postData)

In [338]:
print(dataGetter.text)

<?xml version="1.0" encoding="UTF-8"?>
<CubeView lang="eng"><Data rows="1" cols="24" rowPos="0" colPos="0" rowspan="1" colspan="24" d="6"><Headers rowspan="1" colspan="24"><Slicer dimCount="4"><Dim active="0" dimLabel="PRODUCT" hasDimNotes="false" hasMemberNotes="false" isChild="false" isParent="false" label="Total all products" members="123" position="2" required="true"/><Dim active="0" dimLabel="PARTNER" hasDimNotes="false" hasMemberNotes="false" isChild="true" isParent="false" label="          China, Taiwan Province of" members="1" position="1" required="true"/></Slicer><ColHeader coldimcount="1"><ColDim position="3"><DimLabel>YEAR</DimLabel><ColLabels><ColLabel col="0" member="0" id="L0" isParent="false" isChild="false">1995</ColLabel><ColLabel col="1" member="1" id="L1" isParent="false" isChild="false">1996</ColLabel><ColLabel col="2" member="2" id="L2" isParent="false" isChild="false">1997</ColLabel><ColLabel col="3" member="3" id="L3" isParent="false" isChild="false">1998</ColLa

In [307]:
result = ET.fromstring(dataGetter.text)
target = list()
for value in result.findall(".//C"):
    print(int("".join(value.attrib["f"].split()))) # get rid of space
print(result.tag)

483185
643412
814468
212626
682325
756627
805997
817689
749168
890569
935016
968729
1139387
1401386
1120643
1442842
1843298
2081513
2223616
2306504
2076425
2272040
2568269
3503873
CubeView


In [308]:
ord('A')

65

In [83]:
with open("result.xml", "r", encoding = "utf-8") as file:
    diff = list(difflib.context_diff(dataGetter.text, file.read()))
    if(diff != []):
        with open("delta.txt", "w", encoding = "utf-8") as deltaFile:
            print("".join(diff))

*** 
--- 
***************
*** 36,42 ****
  
  <  C--- 36,42 ----
  "  ?  >! 
  
  <  C***************
*** 147029,147033 ****
  
--- 147029,147033 ----
  e  w  >  
+ 



In [ ]:
print(dataGetter.text)

In [70]:
with open("result.xml", "w", encoding = "utf-8") as file:
    file.write(dataGetter.text)

In [ ]:
reportView =

In [142]:
root = ET.fromstring(dataGetter.text)
for child in root:
    print(child.tag, child.attrib)

Data {'rows': '156', 'cols': '24', 'rowPos': '0', 'colPos': '0', 'rowspan': '100', 'colspan': '24', 'd': '0'}
Footnotes {}
MissingValues {}


In [193]:
print(root.findall("DimLabel"))

AttributeError: 'xml.etree.ElementTree.Element' object has no attribute 'Data'

In [3]:
# deal with input
ECONOMY = str()
YEAR_START = 1995
Year_end = None
PARTNER = list()
PRODUCT_GROUPS = list()

In [290]:
PADDING = ' ' * 10
wb = Workbook()
export = wb.active
export.title = "科技產品出口"
export["A1"] = "Merchandise trade matrix – product groups, exports in thousands" +
                " of United States dollars, {}"
export["A4"] = "ECONOMY"
export["A6"] = "YEAR"
export["A7"] = "PRODUCT"

wb.save('./test.xlsx')

In [341]:
wb = load_workbook("./haha.xlsx")

In [347]:
for sheet in wb:
    print(sheet.title)

科技產品出口
